In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import seaborn as sns
import keras
from keras.models import Model
from keras.layers import Dense
import keras
from keras.preprocessing import image as kimage
from keras.applications.resnet_v2 import preprocess_input
from keras.applications import resnet_v2
import tensorflow as tf

In [ ]:
train = []
test = []
train_labels = []
test_labels = []
lista = ["class_0",
"class_1","class_2","class_3","class_4","class_5",
         "class_6", "class_7","class_8","class_9"]
fold = ["train/", "test/"]
 
for j in fold:
    for i in range(len(lista)):
        path = "Spectrogram/spect_original/"+j+lista[i]+"/"
        Class=os.listdir(path)
        for a in Class:
          image = Image.open(path+a)
          imResize = image.resize((224,224), Image.ANTIALIAS)
          if (j == "train/"):
          #size_image = image_from_array.resize((height, width))
              train.append(np.asarray(imResize))
              train_labels.append(str(lista[i]))
          else:
              test.append(np.asarray(imResize))
              test_labels.append(str(lista[i]))
X_train = train
y_train = train_labels
X_test = test
y_test = test_labels

In [ ]:
def add_3d(x):
  l = []
  for i in x:
    if i.shape == (224,224): #if the image is gray scale shape
      img2 = np.stack((i,i,i), axis = -1) #add three channels
      l.append(img2)
    else:
      l.append(i)
  return np.asarray(l)
X_train = add_3d(X_train)
X_test = add_3d(X_test)

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train,random_state = 1)
X_test, y_test = shuffle(X_test, y_test, random_state = 1)

In [ ]:
train_img_pro = kimage.ImageDataGenerator(preprocessing_function=
resnet_v2.preprocess_input)
test_img_pro = kimage.ImageDataGenerator(preprocessing_function=
resnet_v2.preprocess_input)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
dummy_y = keras.utils.np_utils.to_categorical(y_train)
y_test = encoder.transform(y_test)
dummy_val = keras.utils.np_utils.to_categorical(y_test)

In [ ]:
#Import Resnet model
base_net = resnet_v2.ResNet50V2(input_shape=(224,224,3),
                         include_top=False, #delete last layers
                         weights='imagenet', #imagenet weights 
                         pooling='avg')

In [ ]:
generator_train = train_img_pro.flow(X_train, dummy_y)
generator_test = test_img_pro.flow(X_test, dummy_val, shuffle = False) #pay attention to the shuffle option

In [ ]:
for layer in base_net.layers:
  layer.trainable = False
#Build our NN classificator
x = base_net.output
x = Dense(1024, activation = "relu")(x)
x = Dense(128, activation = "relu")(x)
pred = Dense(10, activation = "softmax")(x)
net = Model(inputs=base_net.input, outputs = pred)
net.compile(loss = keras.losses.categorical_crossentropy, optimizer = "adam", metrics = ['accuracy'])
net.fit_generator(generator_train,validation_data = generator_test, epochs=100)

In [ ]:
net.evaluate_generator(generator_train)

In [ ]:
net.evaluate_generator(generator_test)

In [ ]:
#Plot the confusion matrix
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(20,20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      cm = np.around(cm, decimals=2)
      cm[np.isnan(cm)] = 0.0
      print("Normalized confusion matrix")
      #else:
       # print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Actual Classes')
    plt.xlabel('Predicted Classes')

#Print the Target names
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools 
#shuffle=False
target_names = []

classes = ["0","1","2","3","4","5","6", "7","8","9"]
Y_pred = net.predict_generator(generator_test)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=classes)
disp.plot()
plt.tight_layout()
plt.ylabel('Actual Classes')
plt.xlabel('Predicted Classes')
plt.savefig("confusion_matrix.svg")
plt.show()
print('Classification Report')
print('')
print(classification_report(y_test, y_pred, target_names=classes))

In [ ]:
###############################################################################################################################

In [ ]:
train = []
test = []
train_labels = []
test_labels = []
lista = ["class_0",
"class_1","class_2","class_3","class_4","class_5",
         "class_6", "class_7","class_8","class_9"]
fold = ["train/", "test/"]
 
for j in fold:
    for i in range(len(lista)):
        path = "Spectrogram/spect_original/"+j+lista[i]+"/"
        Class=os.listdir(path)
        for a in Class:
          image = Image.open(path+a)
          imResize = image.resize((224,224), Image.ANTIALIAS)
          #image_from_array = Image.fromarray(image, 'RGB')
          if (j == "train/"):
          #size_image = image_from_array.resize((height, width))
              train.append(np.asarray(imResize))
              train_labels.append(str(lista[i]))
          else:
              test.append(np.asarray(imResize))
              test_labels.append(str(lista[i]))
X_train = train
y_train = train_labels
X_test = test
y_test = test_labels

In [ ]:
def add_3d(x):
  l = []
  for i in x:
    if i.shape == (224,224): #if the image is gray scale shape
      img2 = np.stack((i,i,i), axis = -1) #add three channels
      l.append(img2)
    else:
      l.append(i)
  return np.asarray(l)
X_train = add_3d(X_train)
X_test = add_3d(X_test)

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train,random_state = 1)
X_test, y_test = shuffle(X_test, y_test, random_state = 1)

In [ ]:
train_img_pro = kimage.ImageDataGenerator(preprocessing_function=
resnet_v2.preprocess_input)
test_img_pro = kimage.ImageDataGenerator(preprocessing_function=
resnet_v2.preprocess_input)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
dummy_y = keras.utils.np_utils.to_categorical(y_train)
y_test = encoder.transform(y_test)
dummy_val = keras.utils.np_utils.to_categorical(y_test)

In [ ]:
#Import vgg16 model
from keras.applications.vgg16 import VGG16, preprocess_input
model = VGG16(input_shape = (224, 224, 3),
                 weights='imagenet',
                 include_top=False,
                 pooling = 'avg')

In [ ]:
generator_train = train_img_pro.flow(X_train, dummy_y)
generator_test = test_img_pro.flow(X_test, dummy_val, shuffle = False) #pay attention to the shuffle option

In [ ]:
for layer in model.layers:
  layer.trainable = False
#Build our NN classificator
x = model.output
x = Dense(1024, activation = "relu")(x)
x = Dense(128, activation = "relu")(x)
pred = Dense(10, activation = "softmax")(x)
net = Model(inputs=model.input, outputs = pred)
net.compile(loss = keras.losses.categorical_crossentropy, optimizer = "adam", metrics = ['accuracy'])
net.fit_generator(generator_train,validation_data = generator_test, epochs=100)

In [ ]:
net.evaluate_generator(generator_train)

In [ ]:
net.evaluate_generator(generator_test)

In [ ]:
#Plot the confusion matrix
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(20,20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      cm = np.around(cm, decimals=2)
      cm[np.isnan(cm)] = 0.0
      print("Normalized confusion matrix")
      #else:
       # print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Actual Classes')
    plt.xlabel('Predicted Classes')

#Print the Target names
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools 
#shuffle=False
target_names = []

classes = ["0","1","2","3","4","5","6", "7","8","9"]
Y_pred = net.predict_generator(generator_test)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=classes)
disp.plot()
plt.tight_layout()
plt.ylabel('Actual Classes')
plt.xlabel('Predicted Classes')
plt.savefig("confusion_matrix.svg")
plt.show()
print('Classification Report')
print('')
print(classification_report(y_test, y_pred, target_names=classes))